In [ ]:
import pandas as pd
import os

# Read the nearby_stations.csv file
nearby_stations = pd.read_csv('data/nearby_stations.csv')

# Group by city and find the station with the largest number of records
results = []

for city in nearby_stations['CITY'].unique():
    city_stations = nearby_stations[nearby_stations['CITY'] == city]
    max_records = 0
    max_station_id = None
    
    for station_id in city_stations['ID']:
        filename = f'data/by_station/{station_id}.csv'
        if os.path.exists(filename):
            record_count = len(pd.read_csv(filename))
            if record_count > max_records:
                max_records = record_count
                max_station_id = station_id
    
    if max_station_id is not None:
        results.append({
            'city': city,
            'station_id': max_station_id,
            'records': max_records
        })

# Create a dataframe with the stations that have maximum records for each city
station_ids_with_max_records = [result['station_id'] for result in results]
city_stations_df = nearby_stations[nearby_stations['ID'].isin(station_ids_with_max_records)]

# Write to CSV
city_stations_df.to_csv('data/city_stations.csv', index=False)

# Print results
for result in results:
    print(f"{result['city']}: {result['station_id']}.csv with {result['records']} records")

In [ ]:
import json
import psycopg2
# Open a connection to the database
conn = psycopg2.connect(
    host='sayulita.local',
    database='mcpdb',
    user='postgres',
    password='aiagent'
)
city_stations_df = pd.read_csv('data/city_stations.csv')
# Read the CSV file for all the stations
for station_id in city_stations_df['ID']:
    station_data = pd.read_csv(f'data/by_station/{station_id}.csv')

    # Group by the second column (date) and create JSON objects
    num_records = 0
    for date in station_data.iloc[:, 1].unique():
        date_records = station_data[station_data.iloc[:, 1] == date]
        json_list = []
        
        for _, row in date_records.iterrows():
            key = row.iloc[2]  # 3rd column (index 2)
            value = row.iloc[3]  # 4th column (index 3)
            json_list.append({key: value})
        
        num_records += 1
        json_output = json.dumps(json_list)

        # Insert into the database
        # Convert date from integer format (YYYYMMDD) to string format (YYYY-MM-DD)
        date_str = str(date)
        formatted_date = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        with conn.cursor() as cursor:
            cursor.execute(
                "INSERT INTO daily (location, station_id, station_name, date, data) VALUES (%s, %s, %s, %s, %s)",
                (
                    city_stations_df[city_stations_df['ID'] == station_id]['CITY'].values[0],
                    station_id,
                    city_stations_df[city_stations_df['ID'] == station_id]['NAME'].values[0],
                    formatted_date,
                    json_output
                )
            )

    conn.commit()
    print(f"Total records for station {station_id}: {num_records}")

conn.close()